In [10]:
import simpy
import numpy as np
from random import expovariate, seed


In [2]:
# global constants
N_PROCESSORS = 1
N_REQUESTS = 2000
MAX_QUEUE_LEN = 100
AVG_REQUEST_INTERVAL = 700
AVG_SERVING_TIME = 100
MAX_TIME = 30000
SEED = 42

payload_balance = {
    30000: {
        0.1: {'a': 700, 'b': 100},
        0.9: {'a': 104, 'b': 100},
    },
    20000: {
        0.1: {'a': 1100, 'b': 100},
        0.9: {'a': 104, 'b': 100},
    },
    10000: {
        0.1: {'a': 450, 'b': 100},
        0.9: {'a': 104, 'b': 100},
    },
    8000: {
        0.1: {'a': 450, 'b': 100},
        0.9: {'a': 102, 'b': 100},
    },
    6000: {
        0.1: {'a': 450, 'b': 100},
        0.9: {'a': 102, 'b': 100},
    }
}

In [3]:
class ProcessMonitor:
    ''' Collects the data about running inside process. 
    
    This class does not pretend to be general by any means.
    '''
    def __init__(self, env, resource):
        self.env = env
        self.resource = resource
        
        self._proc_working_time = 0
        self._serving_time = []
        self._waiting_time = []
        self._entry_time = []
        self._num_rejected = 0
        
    def collect_data(self, n_requests, avg_request_interval, avg_serving_time, max_time):
        self.n_requests = n_requests
        self._avg_request_interval = avg_request_interval
        self._avg_serving_time = avg_serving_time
        
        self.env.process(self._generate_requests())
        self.env.run(until=max_time)
        
        return {
            'w': self._waiting_time, 
            'b': self._serving_time,
            'e': self._entry_time,
            'p': sum(self._serving_time),
            'n_rejected': self._num_rejected,
        }
    
    def _generate_requests(self):
        for i in range(self.n_requests):
            self.env.process(self._serve())
            t = expovariate(1.0/self._avg_request_interval)
            yield self.env.timeout(t) # wait for the next request to appear      
        
    def _serve(self):
        num_rejected = 0
        arrive = self.env.now
        
        self._entry_time.append(arrive)
        
        if len(self.resource.queue) < MAX_QUEUE_LEN:
            with self.resource.request() as req:
                yield req
                wait_time = self.env.now-arrive
                self._waiting_time.append(wait_time)

                service_time = expovariate(1.0/self._avg_serving_time)
                before_service = env.now
                yield self.env.timeout(service_time) # wait to be served
                self._serving_time.append(service_time)

#                 self._proc_working_time += before_service - arrive 
        else:
            num_rejected += 1

In [4]:
seed(SEED)

results = {}

for max_time, payload_params in payload_balance.items():
    results[max_time] = {}
    for payload, values in payload_params.items():
        env = simpy.Environment(0)
        k = simpy.Resource(env, capacity=N_PROCESSORS)
        m = ProcessMonitor(env, k)
        results[max_time][payload] = m.collect_data(N_REQUESTS, values['a'], values['b'], max_time)
        
        print("n={}, a={}, b={}, payload={} p={}".format(max_time, values['a'], values['b'], payload, results[max_time][payload]['p']))

n=30000, a=700, b=100, payload=0.1 p=3288.822499514054
n=30000, a=104, b=100, payload=0.9 p=28452.790393510026
n=20000, a=1100, b=100, payload=0.1 p=3340.52099447941
n=20000, a=104, b=100, payload=0.9 p=19696.726213902944
n=6000, a=450, b=100, payload=0.1 p=1455.7865496932961
n=6000, a=102, b=100, payload=0.9 p=5164.370875097138
n=10000, a=450, b=100, payload=0.1 p=2816.545100785558
n=10000, a=104, b=100, payload=0.9 p=8830.046821486721
n=8000, a=450, b=100, payload=0.1 p=1344.3198124995613
n=8000, a=102, b=100, payload=0.9 p=6244.3383030419645


In [36]:
n_requests = [10, 100, 500, 2000, 4000, 6000, 8000, 10000, 20000, 30000, 50000, 75000, 150000]

results = {}

for nr in n_requests:
    env = simpy.Environment(0)
    k = simpy.Resource(env, capacity=N_PROCESSORS)
    m = ProcessMonitor(env, k)
    results[nr] = m.collect_data(nr, nr/MAX_TIME, 1.11111 * nr/MAX_TIME, MAX_TIME)

In [37]:
lambdas = np.array(n_requests) / MAX_TIME

In [40]:
y = 0.9
p0= (1 - y) 
p = y
    
for nr, _lambda in zip(n_requests, lambdas) :
    ts = (results[nr]['e'])
    payload = results[nr]['p']
    diffs = list(map(lambda i: ts[i+1] - ts[i], range(len(ts) -1)))
    print(nr, p - payload/MAX_TIME, payload/MAX_TIME)

10 0.8999998926304676 1.0736953241133593e-07
100 0.8999883976146932 1.1602385306794327e-05
500 0.8997140347276782 0.00028596527232185485
2000 0.895529509439522 0.004470490560477941
4000 0.881875690813027 0.018124309186973053
6000 0.8592527834320727 0.04074721656792729
8000 0.8283737319885406 0.07162626801145948
10000 0.7885846427557426 0.1114153572442574
20000 0.4520556912162808 0.4479443087837192
30000 -0.0964946372864669 0.9964946372864669
50000 -0.09923851660550165 0.9992385166055017
75000 -0.098780869617217 0.998780869617217
150000 -0.0954708433378767 0.9954708433378767


In [42]:
y = 0.9
p0= (1 - y) 
p = y
params = payload_balance[30000][0.9]
results = {}

lambdas = np.array(n_requests) / MAX_TIME

for nr in n_requests:
    env = simpy.Environment(0)
    k = simpy.Resource(env, capacity=N_PROCESSORS)
    m = ProcessMonitor(env, k)
    results[nr] = m.collect_data(nr, params['a'], params['b'], MAX_TIME)
    
for nr, _lambda in zip(n_requests, lambdas):
    ts = (results[nr]['e'])
    payload = results[nr]['p']
    diffs = list(map(lambda i: ts[i+1] - ts[i], range(len(ts) -1)))
    print(nr, p - payload/MAX_TIME, payload/MAX_TIME)

10 0.8723903069310212 0.02760969306897887
100 0.5458435255766163 0.3541564744233837
500 0.16193562867054256 0.7380643713294575
2000 0.025435268467270955 0.8745647315327291
4000 0.008233887483066304 0.8917661125169337
6000 -0.06284315570246335 0.9628431557024634
8000 -0.08921975609048782 0.9892197560904878
10000 0.16767388186859844 0.7323261181314016
20000 -0.07259066248573831 0.9725906624857383
30000 -0.053488769593987184 0.9534887695939872
50000 -0.012586849081842377 0.9125868490818424
75000 -0.02964669329320746 0.9296466932932075
150000 0.04852577205140696 0.8514742279485931


In [7]:
def grid_search(a_range, b_range, a_or_b, time_):
    grad = 100
    results = {}
    for a in a_range:
        for b in b_range:
            env = simpy.Environment(0)
            k = simpy.Resource(env, capacity=N_PROCESSORS)
            m = ProcessMonitor(env, k)
            results = m.collect_data(N_REQUESTS, a, b, time_)

            last_grad = grad
            grad = (results['p'] / time_) - a_or_b

            if grad < 0.09:
                print(results['p'])
                return (a, b)

            
    return ()
            

def check_payload(results, time_):
    print("Загрузка={}".format(results[time_][0.1]['p'] / time_))
    print("Загрузка={}\n".format(results[time_][0.9]['p'] / time_))


In [8]:
check_payload(results, 30000)

check_payload(results, 20000)
check_payload(results, 10000)
check_payload(results, 8000)
check_payload(results, 6000)

KeyError: 0.1